In [3]:
from tools import *

### Model 4a: Minimize Inbound and Outbound Logistics Constrained on Customer's Demand and Factory Capacity and Sufficient Supply (Multiple Ports) (Lossless Production)

#### Input to the Model: 

1. Number of factories and customers and ports
2. Outbound cost
3. Demands
4. Factory Capacity
5. Inbound cost (each port to each factory)

Because we have multiple ports, we can no longer just add the inbound cost in with the outbound cost. In fact, we have to keep separate inbound volume, and place an additional constraints called sufficient supply to make sure all the outbound volume are inbounded.

In [123]:
no_I, no_J, no_K = 3, 5, 2 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_I, no_K) # Inbound Cost
obij = np.random.rand(no_I, no_J) # Outbound Cost
wj = np.random.rand(no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

In [124]:
def generate_demand_matrix(no_I, no_J, no_K):
    
    Wijk = np.tile(np.hstack([np.zeros((no_J, no_K)), np.eye(no_J)]), reps = no_I)
    
    return Wijk

def generate_capacity_matrix(no_I, no_J, no_K):
    
    factory_block_list = []

    port_block = np.zeros((no_I, no_K))

    for i in range(no_I):
        customer_column = np.zeros((no_I, 1))
        customer_column[i] = 1
        customer_block = np.repeat(customer_column, repeats = no_J, axis = 1)
        factory_block_list.append(np.hstack([port_block, customer_block]))

    Ki = np.concatenate(factory_block_list, axis = 1)
    
    return Ki

In [125]:
## Assume non-trivial

Wijk = generate_demand_matrix(no_I, no_J, no_K) # Demand Constraint Matrix
Ki = generate_capacity_matrix(no_I, no_J, no_K) # Capacity Constraint Matrix

In [1]:
def generate_objective_vector(no_I, no_J, no_K, ibik, obij):
    c = np.hstack([ibik.reshape(no_I, no_K), obij.reshape(no_I, no_J)]).flatten()
    return c

def generate_supply_matrix(no_I, no_J, no_K):
    
    factory_block_list = []

    for i in range(no_I):
        port_column, customer_column = np.zeros((no_I, 1)), np.zeros((no_I, 1))
        port_column[i], customer_column[i] = -1, 1
        
        port_block = np.repeat(port_column, repeats = no_K, axis = 1)
        customer_block = np.repeat(customer_column, repeats = no_J, axis = 1)
        factory_block_list.append(np.hstack([port_block, customer_block]))

    Si = np.concatenate(factory_block_list, axis = 1)
    
    return Si

In [130]:
Si = generate_supply_matrix(no_I, no_J, no_K)

In [2]:
## Standard form of our model

# New cost vector
c = generate_objective_vector(no_I, no_J, no_K, ibik, obij)

# Upper Bound
A_ub = np.vstack([-Wijk, Ki, Si])
b_ub = np.hstack([-wj, ki, np.zeros(no_I)])

prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

NameError: name 'no_I' is not defined

In [146]:
def optimize_logistics_4a(no_I, no_J, no_K, ibik, obij, wj, ki):
    
    ## Assume non-trivial
    Wijk = generate_demand_matrix(no_I, no_J, no_K) # Demand Constraint Matrix
    Ki = generate_capacity_matrix(no_I, no_J, no_K) # Capacity Constraint Matrix

    # Sufficient Supply Matrix
    # Shape is |I| * (|J| + |K|) columns (variables)
    Si = generate_supply_matrix(no_I, no_J, no_K)

    ## Standard form of our model

    # New cost vector
    c = np.hstack([ibik.reshape(no_I, no_K), obij.reshape(no_I, no_J)]).flatten()

    # Upper Bound
    A = np.vstack([-Wijk, Ki, Si])
    b = np.hstack([-wj, ki, np.zeros(no_I)])

    prog = linprog(c, A_ub = A, b_ub = b) # Bigger than or equal constraints

    return prog

In [147]:
no_I, no_J, no_K = 3, 5, 2 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_I, no_K) # Inbound Cost
obij = np.random.rand(no_I, no_J) # Outbound Cost
wj = np.random.rand(no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity
    
prog = optimize_logistics_4a(no_I, no_J, no_K, ibik, obij, wj, ki) # Results are all correct

### Model 4b: Minimize Inbound and Outbound Logistics Constrained on Customer's Demand and Factory Capacity and Sufficient Supply (Multiple Ports) (Lossy Production)

In the case of lossy production, we have a vector that defines the percentage loss per factory.

#### Input to the Model: 

1. Number of factories and customers and ports
2. Outbound cost
3. Inbound cost (each port to each factory)
4. Demands
5. Factory Capacity
6. Loss Percentage / Efficiency

In [215]:
no_I, no_J, no_K = 3, 6, 2 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_I, no_K) # Inbound Cost
obij = np.random.rand(no_I, no_J) # Outbound Cost
wj = np.random.rand(no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity
ei = np.random.rand(no_I) # Efficiency

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

In [220]:
## Assume non-trivial

Wijk = generate_demand_matrix(no_I, no_J, no_K) # Demand Constraint Matrix
Ki = generate_capacity_matrix(no_I, no_J, no_K) # Capacity Constraint Matrix

In [222]:
def generate_supply_matrix_with_efficiency(no_I, no_J, no_K, ei):
    
    factory_block_list = []

    for i in range(no_I):
        port_column, customer_column = np.zeros((no_I, 1)), np.zeros((no_I, 1))
        port_column[i], customer_column[i] = -ei[i], 1
        
        port_block = np.repeat(port_column, repeats = no_K, axis = 1)
        customer_block = np.repeat(customer_column, repeats = no_J, axis = 1)
        factory_block_list.append(np.hstack([port_block, customer_block]))

    Si = np.concatenate(factory_block_list, axis = 1)
    
    return Si

In [224]:
Si = generate_supply_matrix_with_efficiency(no_I, no_J, no_K, ei)

## Standard form of our model

# New cost vector
c = np.hstack([ibik.reshape(no_I, no_K), obij.reshape(no_I, no_J)]).flatten()

# Upper Bound
A_ub = np.vstack([-Wijk, Ki, Si])
b_ub = np.hstack([-wj, ki, np.zeros(no_I)])

prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

In [227]:
Matrix(prog_ub.x.reshape(no_I, no_J+no_K))

Matrix([
[8.96137332988318e-10,  1.83924478015982, 9.56227259479553e-10,  0.246904507858376,  2.76123286786584e-9,  0.0987984506807339, 8.37660753923659e-10,    0.652317111439655],
[ 3.2582845460748e-10,  1.57931813746033, 3.61956716772363e-10,  0.638988129448889,  0.00818964443333091,  1.1883226390292e-9,     0.32199467741788, 4.66163562387438e-10],
[6.47879340119892e-10, 0.646447635695113,     0.43394868265928, 0.0982062539184325, 6.34953356118594e-11, 6.2177694155299e-10,  7.96545143854714e-9,  1.02986567370328e-9]])

In [234]:
### Check demand
aae(np.sum(prog_ub.x.reshape(no_I, no_J+no_K)[:, no_K:], axis = 0), wj)

In [236]:
### Check weighted inbound volume
np.sum(prog_ub.x.reshape(no_I, no_J+no_K)[:, :no_K], axis = 1)*ei

array([0.99802007, 0.96917245, 0.53215494])

In [238]:
### Check outbound volume
np.sum(prog_ub.x.reshape(no_I, no_J+no_K)[:, no_K:], axis = 1)

array([0.99802007, 0.96917245, 0.53215495])

In [240]:
def optimize_logistics_4b(no_I, no_J, no_K, ibik, obij, wj, ki, ei):
    
    assert np.sum(ki) >= np.sum(wj), 'More Demand than Capacity. Program is Impossible'
    
    ## Assume non-trivial
    Wijk = generate_demand_matrix(no_I, no_J, no_K) # Demand Constraint Matrix
    Ki = generate_capacity_matrix(no_I, no_J, no_K) # Capacity Constraint Matrix
    
    # Sufficient Supply Matrix
    Si = generate_supply_matrix_with_efficiency(no_I, no_J, no_K, ei)

    ## Standard form of our model

    # New cost vector
    c = np.hstack([ibik.reshape(no_I, no_K), obij.reshape(no_I, no_J)]).flatten()

    # Upper Bound
    A_ub = np.vstack([-Wijk, Ki, Si])
    b_ub = np.hstack([-wj, ki, np.zeros(no_I)])

    prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints
    
    ### Check weighted inbound volume
    Vin = np.sum(prog_ub.x.reshape(no_I, no_J+no_K)[:, :no_K], axis = 1)*ei
    
    ### Check outbound volume
    Vout = np.sum(prog_ub.x.reshape(no_I, no_J+no_K)[:, no_K:], axis = 1)
    
    aae(Vin, Vout) # Check sufficient supply
    
    return prog